# Exercise 2: Unconstrained Optimization I

## Imports

In [22]:
import numpy as np

## Functions

In [23]:
def f(x):
    """Function to minimize: f(x, y) = (x+1)^2 - xy + 3(y-5)^2"""
    return (x[0] + 1) ** 2 - x[0] * x[1] + 3 * (x[1] - 5) ** 2


def grad_f(x):
    """Gradient of f"""
    dx = 2 * (x[0] + 1) - x[1]
    dy = -x[0] + 6 * (x[1] - 5)
    return np.array([dx, dy])


def hessian_f(x):
    """Hessian of f"""
    return np.array([[2, -1], [-1, 6]])


def hessian_inv_f(x):
    """Inverse Hessian of f (constant for this quadratic)"""
    H = hessian_f(x)
    return np.linalg.inv(H)

## Analytical Solution (grad_f = 0)

In [24]:
def find_optimal_analytical():
    """Find optimal point by solving grad_f = 0
    grad_f = [2(x+1) - y, -x + 6(y-5)] = [0, 0]
        2x + 2 - y = 0  =>  y = 2x + 2
        -x + 6y - 30 = 0  =>  6y = x + 30
    Substitute: 6(2x + 2) = x + 30
        12x + 12 = x + 30
        11x = 18
        x = 18/11
    """

    x_opt = 18 / 11
    y_opt = 2 * x_opt + 2

    optimal_point = np.array([x_opt, y_opt])
    optimal_value = f(optimal_point)

    print("Analytical Solution:")
    print(f"\tOptimal point: ({x_opt:.6f}, {y_opt:.6f})")
    print(f"\tOptimal value: {optimal_value:.6f}")
    print(f"\tGradient at optimal: {grad_f(optimal_point)}")
    print(f"\tHessian:\n{hessian_f(optimal_point)}")

    # Check if Hessian is positive definite
    eigenvalues = np.linalg.eigvals(hessian_f(optimal_point))
    print(f"\tHessian eigenvalues: {eigenvalues}")
    print(f"\tPositive definite: {np.all(eigenvalues > 0)}")

    return optimal_point, optimal_value

## Line search

In [25]:
def armijo_line_search(x_k, grad_k, f_func):
    """Armijo's method for step length selection"""
    alpha = 1.0
    grad_norm_sq = grad_k @ grad_k

    def w(t_val):
        return x_k - t_val * grad_k

    f_xk = f_func(x_k)

    # Check initial condition
    if f_func(w(alpha)) <= f_xk - 0.2 * alpha * grad_norm_sq:
        # Need to increase alpha
        while f_func(w(alpha)) <= f_xk - 0.2 * alpha * grad_norm_sq:
            t_prev = alpha
            alpha = 2 * alpha
            if alpha > 1e6:
                return t_prev
        return t_prev
    else:
        # Need to decrease alpha
        while f_func(w(alpha)) > f_xk - 0.2 * alpha * grad_norm_sq:
            alpha = alpha / 2
            if alpha < 1e-10:
                return alpha
        return alpha

## Steepest Descent

In [26]:
def steepest_descent(x0, tol=1e-3, max_iter=10000):
    """Steepest descent method with Armijo line search"""
    x_k = x0.copy()
    iterations = 0
    points = [x_k.copy()]

    while True:
        grad_k = grad_f(x_k)
        grad_norm = np.linalg.norm(grad_k)

        if grad_norm < tol:
            break

        if iterations >= max_iter:
            print(f"Warning: Maximum iterations ({max_iter}) reached")
            break

        t_k = armijo_line_search(x_k, grad_k, f)

        # Update
        x_k = x_k - t_k * grad_k
        points.append(x_k.copy())
        iterations += 1

    return x_k, iterations, points

## Newton's Method

In [27]:
def newtons_method(x0, tol=1e-3, max_iter=100):
    """Newton's method for optimization"""
    x_k = x0.copy()
    iterations = 0
    points = [x_k.copy()]

    while True:
        grad_k = grad_f(x_k)
        grad_norm = np.linalg.norm(grad_k)

        if grad_norm < tol:
            break

        if iterations >= max_iter:
            print(f"Warning: Maximum iterations ({max_iter}) reached")
            break

        # Newton update
        H_inv = hessian_inv_f(x_k)
        x_k = x_k - H_inv @ grad_k
        points.append(x_k.copy())
        iterations += 1

    return x_k, iterations, points

## Main

In [28]:
# Analysis
x_optimal, f_optimal = find_optimal_analytical()


# Steepest Descent
x0 = np.array([1.0, 1.0])
print("\nSteepest Descent Method:")
x_sd, iter_sd, points_sd = steepest_descent(x0)
print(f"\tStarting point: ({x0[0]:.6f}, {x0[1]:.6f})")
print(f"\tFinal point: ({x_sd[0]:.6f}, {x_sd[1]:.6f})")
print(f"\tNumber of iterations: {iter_sd}")
print(f"\tFinal function value: {f(x_sd):.6f}")
print(f"\tOptimal function value: {f_optimal:.6f}")
print(f"\tAbsolute error: {abs(f(x_sd) - f_optimal):.6e}")
print(f"\tFinal gradient norm: {np.linalg.norm(grad_f(x_sd)):.6e}")


# Newton's Method
print("\nNewton's Method:")
x_nm, iter_nm, points_nm = newtons_method(x0)
print(f"\tStarting point: ({x0[0]:.6f}, {x0[1]:.6f})")
print(f"\tFinal point: ({x_nm[0]:.6f}, {x_nm[1]:.6f})")
print(f"\tNumber of iterations: {iter_nm}")
print(f"\tFinal function value: {f(x_nm):.6f}")
print(f"\tOptimal function value: {f_optimal:.6f}")
print(f"\tAbsolute error: {abs(f(x_nm) - f_optimal):.6e}")
print(f"\tFinal gradient norm: {np.linalg.norm(grad_f(x_nm)):.6e}")

Analytical Solution:
	Optimal point: (1.636364, 5.272727)
	Optimal value: -1.454545
	Gradient at optimal: [0.00000000e+00 3.77475828e-15]
	Hessian:
[[ 2 -1]
 [-1  6]]
	Hessian eigenvalues: [1.76393202 6.23606798]
	Positive definite: True

Steepest Descent Method:
	Starting point: (1.000000, 1.000000)
	Final point: (1.636346, 5.272606)
	Number of iterations: 18
	Final function value: -1.454545
	Optimal function value: -1.454545
	Absolute error: 4.237244e-08
	Final gradient norm: 7.156404e-04

Newton's Method:
	Starting point: (1.000000, 1.000000)
	Final point: (1.636364, 5.272727)
	Number of iterations: 1
	Final function value: -1.454545
	Optimal function value: -1.454545
	Absolute error: 0.000000e+00
	Final gradient norm: 3.552714e-15
